<a class="reference external" 
    href="https://jupyter.designsafe-ci.org/hub/user-redirect/lab/tree/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Jupyter_Notebooks/paths_InPython_Manage.ipynb" target="_blank">
<img alt="Try on DesignSafe" src="https://raw.githubusercontent.com/DesignSafe-Training/pinn/main/DesignSafe-Badge.svg" /></a>

# Managing Paths 📒
***Managing and Navigating Files & Directories***

by Silvia Mazzoni, DesignSafe, 2025

Learn how to navigate, organize, and automate your filesystem in Python — from checking your current path to creating, moving, and cleaning up files and folders.

Working with files and directories is a core part of any Python project.
Whether you’re inspecting your current location, creating or removing folders, navigating between directories, or performing higher-level operations like copying and moving with *shutil*, Python’s standard library provides everything you need to manage your filesystem directly from code.

In this section, we’ll explore practical tools and patterns for:

* **Finding your current working directory**
* **Creating and removing directories**
* **Changing your working location**
* **Splitting and joining paths**
* **Using *shutil* for file and directory operations**
* **Separating files from folders**

By mastering these techniques, you can automate data organization, build reproducible workflows, and handle file I/O tasks without relying on manual file management.



In [1]:
import os

---
## Getting your current path

Because different environments mount storage systems differently, querying the current path (**"."**) may show different results.

To get the **absolute path of your current directory**, you can use:

In [2]:
os.getcwd()

'/home/jupyter/MyData/_ToCommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Jupyter_Notebooks'

In [3]:
os.path.abspath('.')           # same result, more general

'/home/jupyter/MyData/_ToCommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Jupyter_Notebooks'

Both will typically give the same absolute path. In this notebook, we’ll change directories into each storage system (and a subfolder) and query **"."** to illustrate how paths differ across environments.

---
## Creating Directories

When working with file outputs or organizing your data, it’s common to need a specific directory. Rather than assuming it exists, your script can **check and create it if necessary**.

Use **os.path.exists()** to check whether a directory is already present, and **os.makedirs()** to create it if it’s missing.

In [4]:
thisPath = 'results/output_data'

if not os.path.exists(thisPath):
    print(f"Creating missing directory: {thisPath}")
    os.makedirs(thisPath)
else:
    print(f"Directory already exists: {thisPath}")

Directory already exists: results/output_data


### Why use *os.makedirs()*?

* It **creates intermediate directories** as needed (e.g., *a/b/c* even if *a* and *b* don’t exist yet).
* It’s safe to use in automated workflows where directory structure might vary.
* Use it **before writing files** to ensure your destination exists.

> ✅ Tip: If you use Python 3.2 or newer, you can also add `exist_ok=True` to skip the existence check:

In [5]:
thisPath = 'temporalxxx'
os.makedirs(thisPath, exist_ok=True); 

This makes your script **more robust and portable**, especially in shared or multi-user environments like DesignSafe.

---
## Deleting Files and Directories
***deleting files and directories safely using Python***

Sometimes you’ll want to **clean up old files**, remove temporary data, or ensure a directory is empty before writing new output. Python provides built-in tools for this, mostly through the *os* and *shutil* modules.

### Deleting a file

In [6]:
file_path = 'output/log.txt'

if os.path.exists(file_path):
    os.remove(file_path)
    print(f"Deleted file: {file_path}")
else:
    print(f"File not found: {file_path}")

File not found: output/log.txt


### Deleting an Empty Directory

Use **os.rmdir()** to delete a directory **only if it's empty**:

In [7]:
thisPath = 'temporalxxx'
if os.path.exists(thisPath):
    try:
        os.rmdir(thisPath)
        print(f"Deleted empty directory: {thisPath}")
    except OSError:
        print(f"Directory not empty: {thisPath}")

Deleted empty directory: temporalxxx


### Deleting a Directory and All Its Contents

To delete a directory **and everything inside it**, use **shutil.rmtree()**:

In [8]:
import shutil

dir_path = 'results/temp'

if os.path.exists(dir_path):
    shutil.rmtree(dir_path)
    print(f"Deleted directory and all contents: {dir_path}")
else:
    print(f"Directory not found: {dir_path}")

Directory not found: results/temp


> ⚠️ **Warning:** *shutil.rmtree()* is **not reversible**. Always double-check the path before using it to avoid deleting critical data.

---

This gives you full control over cleaning up files and directories in your workflow — whether you're preparing for a new simulation run or automating post-processing steps.


## Why Use *shutil*?

While **os** and **os.path** handle **basic file and path operations**, the *shutil*` module provides **higher-level utilities** for working with files and directories — especially when you need to:

* **Copy** files or folders
* **Delete entire directories** (recursively)
* **Move or rename** large data structures

It's a powerful tool for scripting workflows that manipulate lots of data, such as preparing input folders, archiving results, or cleaning up temporary files.

---

### Common *shutil* Functions

| Task                             | Python Code                         | Description                                     |
| -------------------------------- | ----------------------------------- | ----------------------------------------------- |
| Copy a file                      | *shutil.copy('src.txt', 'dst.txt')* | Copies the contents and metadata of a file      |
| Copy a directory (recursively)   | *shutil.copytree('src/', 'dst/')*   | Copies a full folder structure and contents     |
| Move or rename a file/folder     | *shutil.move('old', 'new')*         | Moves or renames files or folders               |
| Delete a directory (recursively) | *shutil.rmtree('folder')*           | Removes a directory and all its contents        |
| Disk usage of a path             | *shutil.disk_usage('/')*            | Shows total, used, and free space at a location |
| Create archive (zip/tar)         | *shutil.make_archive(...)*          | Builds a *.zip*, *.tar*, or *.gztar* archive    |

---

✅ **Note:** *shutil* is part of the Python Standard Library — no installation needed!


---
## Changing Directories
***Changing your working directory with os.chdir***

Once you’ve found the path you want to work in, you can **change your current working directory** so that all relative operations happen there.  
This is done using Python’s:

In [9]:
print('current location',os.getcwd())

thisPath = os.path.expanduser('~/MyData')
os.chdir(thisPath)

print('new location',os.getcwd())

current location /home/jupyter/MyData/_ToCommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Jupyter_Notebooks
new location /home/jupyter/MyData


From this point on, any command using a relative path (like *"./somefile.txt"*) will look inside this directory.

### Always check that the directory exists first

Before changing directories, it’s good practice to make sure the path actually exists.
Otherwise, *os.chdir* will throw a *FileNotFoundError*.

In [10]:
print('current location',os.getcwd())

thisPath = os.path.abspath(os.path.expanduser('~/YourData'))

if os.path.exists(thisPath):
    os.chdir(thisPath)
    print(f"Changed working directory to: {thisPath}")
    print(f"Current working directory is now: {os.getcwd()}")
else:
    print(f"Path does not exist: {thisPath}")
    
print('new location',os.getcwd())

current location /home/jupyter/MyData
Path does not exist: /home/jupyter/YourData
new location /home/jupyter/MyData


### Why change directories?

* Lets you keep your relative paths simple.
* Avoids having to write out long absolute paths repeatedly.
* Makes it easier to run scripts or load data, knowing they’ll look in this directory.

### A few tips

* Always print your new location with **os.getcwd()** after changing directories — this avoids confusion about where your notebook or script is “pointing.”
* Remember this change is **per-process**: it only affects the current Python session.

Using *os.chdir* along with *os.path.exists*, *os.getcwd*, and your robust path-building ensures your notebooks are **safe, portable, and clear across all DesignSafe platforms.**


This prints a simple list of names (both files and directories) found under *thisPath*.